In [10]:
import pickle
import gtsam
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import KDTree

from utils import get_points, get_all_context

In [2]:
with open(r"/home/jake/Desktop/holoocean_bags/scrape/1.pickle", "rb") as input_file:
    data_one = pickle.load(input_file)
    
with open(r"/home/jake/Desktop/holoocean_bags/scrape/2.pickle", "rb") as input_file:
    data_two = pickle.load(input_file)

In [3]:
poses_one = data_one["poses"]
points_one = data_one["points"]
points_t_one = data_one["points_t"]

poses_two = data_two["poses"]
points_two = data_two["points"]
points_t_two = data_two["points_t"]

In [29]:
SUBMAP_SIZE = 1
BEARING_BINS = 100
RANGE_BINS = 16
MAX_RANGE = 40
MAX_BEARING = 180
KNN = 5
MAX_TREE_DIST = 20

In [8]:
keys_one, context_one = get_all_context(poses_one,
                                           points_one,
                                           SUBMAP_SIZE,
                                           BEARING_BINS,
                                           RANGE_BINS,
                                           MAX_RANGE,
                                           MAX_BEARING)

keys_two, context_two = get_all_context(poses_two,
                                           points_two,
                                           SUBMAP_SIZE,
                                           BEARING_BINS,
                                           RANGE_BINS,
                                           MAX_RANGE,
                                           MAX_BEARING)

In [30]:
tree_one = KDTree(keys_one)
tree_two = KDTree(keys_one)

for i,key in enumerate(keys_one):
    distances, indexes = tree_two.query(key,k=KNN,distance_upper_bound=MAX_TREE_DIST)
    
for i,key in enumerate(keys_two):
    distances, indexes = tree_one.query(key,k=KNN,distance_upper_bound=MAX_TREE_DIST)